# Forecasting Sales

In [1]:
# Standard Packages
import pandas as pd
import numpy as np
import warnings
import re
import time

# Viz Packages
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 

# Modeling Packages
## Modeling Prep
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, KFold, \
GridSearchCV, RandomizedSearchCV

## SKLearn Data Prep Modules
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, \
PolynomialFeatures, PowerTransformer, Normalizer, MaxAbsScaler
from sklearn.impute import SimpleImputer

## SKLearn Random Forest
from sklearn.ensemble import RandomForestClassifier

## SKLearn Pipeline and Transformer
from imblearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

## SKLearn Model Optimization
from sklearn.feature_selection import RFE, f_regression

## SKLearn Metrics
from sklearn.metrics import mean_absolute_error as MAE, mean_squared_error as MSE

## Statsmodel Time Series modules
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.api import SARIMAX, AutoReg
from statsmodels.tsa.stattools import adfuller

## Holidays package
from datetime import date
import holidays

In [2]:
# Notebook Config
## Suppress Python Warnings (Future, Deprecation)
warnings.filterwarnings("ignore", category= FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)

## Suppress Pandas Warnings (SettingWithCopy)
pd.options.mode.chained_assignment = None

## Pandas Display Config
pd.options.display.max_columns = None
pd.options.display.width = None

## Display SKLearn estimators as diagrams
from sklearn import set_config
set_config(display= 'diagram')

## Read in and Inspect the provided datasets

In [3]:
# Load the provided datasets
stores_df = pd.read_csv('data/stores.csv')
features_df = pd.read_csv('data/features.csv')
test_df = pd.read_csv('data/test.csv')
train_df = pd.read_csv('data/train.csv')

In [4]:
# inspect stores df
stores_df.head()

In [5]:
stores_df.info() # no missing values, convert type to nominal or ordinal?

In [6]:
stores_df['Type'].value_counts()

In [7]:
# Inspect the features data
features_df.head()

In [8]:
features_df.info() # Missing markdown values as expected, some nulls for CPI and unemployment as well
# Need to convert 'Date' to datetime and 'IsHoliday' to numerical bool

In [9]:
features_df.isna().sum()

In [10]:
# Inspect the provided datasets
train_df.head()

In [11]:
train_df.info() # Has the most values - merge stores and features onto this dataset

In [12]:
# Inspect the test data
test_df.head() # will need to also merge the stores and features datasets to our test set?

In [13]:
test_df.info() # roughly 80/20 train/test split

In [14]:
# Combine datasets
train_df = train_df.merge(features_df, how='left').merge(stores_df, how='left')
train_df

In [15]:
# Reorder columns to group similar features 
cols = list(train_df.columns.values)
cols # move 'Type' and 'Size' to after 'Dept', 'CPI' and 'Unemployment' after 'Fuel_Price'

In [16]:
# rewrite df with new column ordering
train_df = train_df[['Store','Dept','Type','Size','Date','Weekly_Sales','IsHoliday','Temperature',
                         'Fuel_Price','CPI','Unemployment','MarkDown1','MarkDown2','MarkDown3','MarkDown4',
                         'MarkDown5']]

In [17]:
# Combine datasets for testing set to mirror training set
test_df = test_df.merge(features_df, how='left').merge(stores_df, how='left')
test_df

In [18]:
# add column to hold predicted weekly sales values 
test_df['Weekly_Sales'] = ''

In [19]:
# reorder columns to mirror transformation to training df
test_df = test_df[['Store','Dept','Type','Size','Date','Weekly_Sales','IsHoliday','Temperature',
                         'Fuel_Price','CPI','Unemployment','MarkDown1','MarkDown2','MarkDown3','MarkDown4',
                         'MarkDown5']]

In [20]:
test_df.info()

## EDA

In [21]:
# check for nulls and dytpes again
train_df.info() # no nulls in CPI and Unemployment now, will need to see what's going there

In [22]:
train_df['CPI'].plot()

In [23]:
train_df['Unemployment'].plot()

In [24]:
train_df.isna().sum() # no more missing values - different reporting windows

In [25]:
# Visualize feature distributions excl markdown and sales cols with scatter plots
cols_to_scatter = ['Store','Dept','Type','Size','Date','IsHoliday','Temperature',
                   'Fuel_Price','CPI','Unemployment']
for col in cols_to_scatter:
    plt.figure()
    plt.scatter(train_df[col], train_df['Weekly_Sales'])
    plt.xlabel(col)
    plt.ylabel('Weekly_Sales')
    plt.figure(figsize=(6,4))

While some type 'B' and midsize stores have sales numbers bigger than some type 'A' and larger stores, we'll handle type ordinally
- big drop off when store type is 'C'

Two huge sales bumps in the date plot
- guessing its thanksgiving/black friday and christmas, but we'll want to drill down on that later 


In [26]:
# Check correlation of numerical features
plt.figure(figsize=(20,10))
cor = train_df.corr()
sns.heatmap(cor, annot=True)
plt.show()

- nothing really jumps out as being highly correlated
- unexpectedly low correlation for many features such as markdowns and external factors such as CPI, gas prices, and the unemployment rate
- could be a case where the best predictor of future values is truly past values

In [27]:
# Extract years, months, weeks from the date and weekly sales into new cols for testing and training dfs
# allows us to group sales by diffrent windows and plot so we can look for trends
# Also necessary for running models later on
# Convert date col to datetime object for training and test sets
train_df['Date'] = pd.to_datetime(train_df['Date'])
test_df['Date'] = pd.to_datetime(test_df['Date'])

# Extract Years, Months, and Weeks from date col in both sets
train_df['Year'] = train_df['Date'].dt.year
train_df['Month'] = train_df['Date'].dt.month
train_df['Week'] = train_df['Date'].dt.week

test_df['Year'] = test_df['Date'].dt.year
test_df['Month'] = test_df['Date'].dt.month
test_df['Week'] = test_df['Date'].dt.week

In [28]:
# Aggregate average sales by year by week to indentify trends in weekly sales 
weekly_2010 = train_df.loc[train_df['Year'] == 2010].groupby('Week')['Weekly_Sales'].mean().reset_index()
weekly_2011 = train_df.loc[train_df['Year'] == 2011].groupby('Week')['Weekly_Sales'].mean().reset_index()
weekly_2012 = train_df.loc[train_df['Year'] == 2012].groupby('Week')['Weekly_Sales'].mean().reset_index()

In [33]:
# Plot weekly sales 
sns.lineplot(weekly_2010)
# plt.plot(weekly_2011)
# plt.plot(weekly_2012)

## Save this for the propeht modeling which takes a holiday argument

In [30]:
# get list of US holidays from the holidays python package for the applicable years (2010,2011,2012)
for date, name in sorted(holidays.US(years=[2010,2011,2012]).items()):
    print(date, name)

Thanksgiving and Black Friday (tgiving +1) always fall in november
- could categorize month 11 as 'thanksgiving' or assign thanksgiving and black friday to the appropriate week number

New Years is technically always in a new week/month/year, however case to be made for grouping with xmas
- could categorize month 12 as xmas & new years or assign to the appropriate week number
- more inclined to encode on monthly basis as trend has been for an ever longer holiday shopping window

Holidays to look at not included in the standard list are:
- Super bowl weekend (
- Valentine's day 
- Easter

Holidays not associated with high levels of consumerism to remove
- Martin Luther King Jr. Day
- Washington's Birthday
- Veterans Day
- Columbus Day
- 'Observed' dates

Other Notes:
- Labor day is not in the test set so we don't need to predict for that
